In [1]:

import pandas as pd
import difflib 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data loading and preproccesing

In [2]:
#loading the csv file
movies_df=pd.read_csv("C:/Users/swapn/Desktop/ml projects/movies.csv")
movies_df

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

In [3]:
#first 5 rows
movies_df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
#columns name
movies_df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [5]:
#No of unique in original_language
movies_df['original_language'].unique()

array(['en', 'ja', 'fr', 'zh', 'es', 'de', 'hi', 'ru', 'ko', 'te', 'cn',
       'it', 'nl', 'ta', 'sv', 'th', 'da', 'xx', 'hu', 'cs', 'pt', 'is',
       'tr', 'nb', 'af', 'pl', 'he', 'ar', 'vi', 'ky', 'id', 'ro', 'fa',
       'no', 'sl', 'ps', 'el'], dtype=object)

In [6]:
 #selecting features
selected_features=['genres','keywords','tagline','cast','director','overview','original_language','production_companies','spoken_languages']
selected_features

['genres',
 'keywords',
 'tagline',
 'cast',
 'director',
 'overview',
 'original_language',
 'production_companies',
 'spoken_languages']

In [7]:
movies_df[['genres','keywords','tagline','cast','director','overview','original_language','production_companies','spoken_languages']].isnull().sum()

genres                   28
keywords                412
tagline                 844
cast                     43
director                 30
overview                  3
original_language         0
production_companies      0
spoken_languages          0
dtype: int64

In [8]:
for i in selected_features:
    movies_df[i]=movies_df[i].fillna('')

In [9]:
#combing the all selected features
combination=movies_df['genres']+' '+movies_df['keywords']+movies_df['tagline']+' '+movies_df['cast']+' '+movies_df['director']+' '+movies_df['overview']+movies_df['original_language']+' '+movies_df['production_companies']+' '+movies_df['spoken_languages']

In [10]:
combination

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance A newlywed couple's honeymoon i...
4800    Comedy Drama Romance TV Movie date love at fir...
4801     A New Yorker in Shanghai Daniel Henney Eliza ...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [11]:
#coverting text to numerical 
vector=TfidfVectorizer()


In [12]:
feature_vector=vector.fit_transform(combination)

In [13]:
print(feature_vector)

  (0, 38707)	0.07966308068391657
  (0, 15962)	0.07941397616050089
  (0, 15934)	0.07941397616050089
  (0, 15706)	0.02354553092176599
  (0, 21802)	0.044887488525746325
  (0, 3950)	0.16942959255367226
  (0, 24106)	0.16942959255367226
  (0, 3343)	0.11866791660056733
  (0, 15777)	0.1024742832561922
  (0, 14950)	0.10948562983242734
  (0, 2599)	0.08972596367099903
  (0, 12320)	0.07576932374475952
  (0, 17598)	0.07657106721743151
  (0, 38404)	0.08875890003936315
  (0, 2478)	0.13394870369262693
  (0, 20859)	0.0948725135699044
  (0, 29054)	0.1097292291702086
  (0, 17000)	0.10852971088906416
  (0, 21331)	0.12653099969786447
  (0, 27266)	0.1340192079029701
  (0, 15613)	0.045908510008860426
  (0, 11239)	0.14521011803125394
  (0, 6183)	0.10948562983242734
  (0, 6454)	0.04667256604672414
  (0, 30679)	0.14780670579991184
  :	:
  (4802, 19975)	0.07746146210144227
  (4802, 19744)	0.1016448970161632
  (4802, 40604)	0.03523429817791381
  (4802, 20194)	0.027887411689059047
  (4802, 27569)	0.039414197523194

cosine similarity

In [14]:
#similarity score using cosine siilarity
similarity=cosine_similarity(feature_vector)

In [15]:
similarity

array([[1.        , 0.06289311, 0.07901597, ..., 0.04123075, 0.03268344,
        0.02183515],
       [0.06289311, 1.        , 0.06343948, ..., 0.05232338, 0.03266722,
        0.03242058],
       [0.07901597, 0.06343948, 1.        , ..., 0.03900811, 0.04921809,
        0.02504503],
       ...,
       [0.04123075, 0.05232338, 0.03900811, ..., 1.        , 0.04385221,
        0.03956803],
       [0.03268344, 0.03266722, 0.04921809, ..., 0.04385221, 1.        ,
        0.03218165],
       [0.02183515, 0.03242058, 0.02504503, ..., 0.03956803, 0.03218165,
        1.        ]])

In [16]:
#to check the shape
similarity.shape

(4803, 4803)

In [17]:
#getting the user name
movie_name=input("Enter the movie name: ")

Enter the movie name: spider man


In [18]:
#creating a list with all the movie name given in the dataset
title_list=movies_df['title'].tolist()
title_list

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [19]:
#finding the close match for the movie name given by the user
close_match=difflib.get_close_matches(movie_name,title_list)
close_match

['Spider-Man', 'Inside Man', 'Superman']

In [20]:
match=close_match[0]
match

'Spider-Man'

In [21]:
#finding the index of the movie with title
index_movie=movies_df[movies_df['title']==match]['index'].values[0]
index_movie

159

In [22]:
#getting a list of similar movies
similarity_score=list(enumerate(similarity[index_movie]))

In [23]:
similarity_score

[(0, 0.047031724327980334),
 (1, 0.03233999032737027),
 (2, 0.04279933263213244),
 (3, 0.04145712121997156),
 (4, 0.052636015465965594),
 (5, 0.3613864289375022),
 (6, 0.019067937484727737),
 (7, 0.04687431498604891),
 (8, 0.02981316001005514),
 (9, 0.02933403428959353),
 (10, 0.05827916986884083),
 (11, 0.030283469745459972),
 (12, 0.0323492384033413),
 (13, 0.029967763269090102),
 (14, 0.04591260501166915),
 (15, 0.036322395561008566),
 (16, 0.05157988461662966),
 (17, 0.024816134097313355),
 (18, 0.018505693761087592),
 (19, 0.03696590698979029),
 (20, 0.14755084703137877),
 (21, 0.021819860353403085),
 (22, 0.03110098870826809),
 (23, 0.040605862427107994),
 (24, 0.034221711079627445),
 (25, 0.020123950910826728),
 (26, 0.05350645585248393),
 (27, 0.03281013671841084),
 (28, 0.03026096467747871),
 (29, 0.034630292274743235),
 (30, 0.3028873912570472),
 (31, 0.07105665833457164),
 (32, 0.03078187038872656),
 (33, 0.09653007294527889),
 (34, 0.015523823302318638),
 (35, 0.04658469373

In [24]:
#sorting the similarity score in descending order
sorted_similarity_movies=sorted(similarity_score,key=lambda x:x[1],reverse=True)
sorted_similarity_movies

[(159, 1.0000000000000002),
 (5, 0.3613864289375022),
 (30, 0.3028873912570472),
 (1534, 0.1996106314510823),
 (38, 0.18039555985774905),
 (20, 0.14755084703137877),
 (431, 0.12223810081526174),
 (511, 0.11467888792526962),
 (122, 0.11183346719296422),
 (767, 0.10691583819793146),
 (37, 0.10678355940233454),
 (203, 0.10655369730526004),
 (1119, 0.10050526618295677),
 (1796, 0.10013367639111388),
 (382, 0.09662531676880821),
 (33, 0.09653007294527889),
 (3265, 0.09551183661786575),
 (1523, 0.09386558210868781),
 (165, 0.09154643039246717),
 (2375, 0.09103562231123098),
 (64, 0.09094521179562981),
 (2361, 0.08981836392031009),
 (953, 0.0897457312320652),
 (668, 0.08904210355136849),
 (259, 0.08827524106340107),
 (2183, 0.08721231272000247),
 (805, 0.08683645323795754),
 (1559, 0.08641666446819862),
 (2163, 0.0858494521210791),
 (94, 0.08386279378735464),
 (101, 0.08315368785313379),
 (677, 0.08236506575571563),
 (3044, 0.08223438530451833),
 (3949, 0.08177013189138267),
 (3124, 0.0804134

In [25]:
#print the similar name based on the index
print('Movies suggested are: \n')
j=1
for i in sorted_similarity_movies:
    index=i[0]
    title_from_index=movies_df[movies_df['index']==index]['title'].values[0]
    if(j<25):
        print(j,' - ',title_from_index)
        j+=1

Movies suggested are: 

1  -  Spider-Man
2  -  Spider-Man 3
3  -  Spider-Man 2
4  -  Arachnophobia
5  -  The Amazing Spider-Man 2
6  -  The Amazing Spider-Man
7  -  Charlotte's Web
8  -  X-Men
9  -  X-Men Origins: Wolverine
10  -  xXx: State of the Union
11  -  Oz: The Great and Powerful
12  -  X2
13  -  21 Jump Street
14  -  Brothers
15  -  Seabiscuit
16  -  X-Men: The Last Stand
17  -  Shadow of the Vampire
18  -  The Good German
19  -  Hulk
20  -  Midnight Special
21  -  X-Men: Apocalypse
22  -  The Ice Storm
23  -  Gremlins 2: The New Batch
24  -  Elektra


In [26]:
movie_name=input("Enter the movie name: ")
title_list=movies_df['title'].tolist()

close_match=difflib.get_close_matches(movie_name,title_list)
match=close_match[0]
index_movie=movies_df[movies_df['title']==match]['index'].values[0]
similarity_score=list(enumerate(similarity[index_movie]))

sorted_similarity_movies=sorted(similarity_score,key=lambda x:x[1],reverse=True)
print('Movies suggested are: \n')
j=1
for i in sorted_similarity_movies:
    index=i[0]
    title_from_index=movies_df[movies_df['index']==index]['title'].values[0]
    if(j<25):
        print(j,' - ',title_from_index)
        j+=1


Enter the movie name: bat man
Movies suggested are: 

1  -  Batman
2  -  Batman Returns
3  -  Batman & Robin
4  -  Batman Begins
5  -  The Dark Knight Rises
6  -  The Dark Knight
7  -  Batman: The Dark Knight Returns, Part 2
8  -  Batman Forever
9  -  Batman v Superman: Dawn of Justice
10  -  Suicide Squad
11  -  The Hudsucker Proxy
12  -  Corpse Bride
13  -  Superman Returns
14  -  A History of Violence
15  -  Green Lantern
16  -  The Spirit
17  -  The Losers
18  -  Big Eyes
19  -  The Clan of the Cave Bear
20  -  Man of Steel
21  -  The Color Purple
22  -  Chill Factor
23  -  Mars Attacks!
24  -  Watchmen
